In [5]:
import scala.collection.JavaConverters._
import java.util.concurrent._
import scala.collection.immutable._
import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global
import org.apache.spark.sql._ 
import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions._

import scala.collection.JavaConverters._
import java.util.concurrent._
import scala.collection.immutable._
import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions._

In [6]:
spark.conf.set(   "fs.azure.account.key.waginput.blob.core.windows.net", "TBwLoPOLim87APX5grtZzWy8Td9h69F/BJgDxuiQyEP480Cs5zyOa2bUeVVRfUnCALOug3aA2Wb4cj8aIiqGEw==")

In [9]:
val dfyellow = spark.read.option("header","true").option("inferSchema","true").parquet("wasbs://opendataset@waginput.blob.core.windows.net/nyctlc/yellow/*/*/")

dfyellow: org.apache.spark.sql.DataFrame = [vendorID: string, tpepPickupDateTime: timestamp ... 19 more fields]

In [15]:
val dfyellow = spark.read.option("header","true").option("inferSchema","true").csv("wasbs://ididims@waginput.blob.core.windows.net/productdim.csv")

dfyellow: org.apache.spark.sql.DataFrame = [eventdatetime: timestamp, customername: string ... 5 more fields]

In [16]:
display(dfyellow)

In [17]:
import java.security.MessageDigest

import java.security.MessageDigest

In [18]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import javax.xml.bind.DatatypeConverter

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import javax.xml.bind.DatatypeConverter

In [19]:
import java.security.MessageDigest
import java.util
import javax.crypto.Cipher
import javax.crypto.spec.SecretKeySpec
import org.apache.commons.codec.binary.Base64
import java.security.DigestException;
import java.security.InvalidAlgorithmParameterException;
import java.security.InvalidKeyException;

import java.security.NoSuchAlgorithmException;
import java.security.SecureRandom;
import java.util.Arrays;
import javax.crypto.BadPaddingException;
import javax.crypto.Cipher;
import javax.crypto.IllegalBlockSizeException;
import javax.crypto.NoSuchPaddingException;
import javax.crypto.spec.IvParameterSpec;
import javax.crypto.spec.SecretKeySpec;

import java.security.MessageDigest
import java.util
import javax.crypto.Cipher
import javax.crypto.spec.SecretKeySpec
import org.apache.commons.codec.binary.Base64
import java.security.DigestException
import java.security.InvalidAlgorithmParameterException
import java.security.InvalidKeyException
import java.security.NoSuchAlgorithmException
import java.security.SecureRandom
import java.util.Arrays
import javax.crypto.BadPaddingException
import javax.crypto.Cipher
import javax.crypto.IllegalBlockSizeException
import javax.crypto.NoSuchPaddingException
import javax.crypto.spec.IvParameterSpec
import javax.crypto.spec.SecretKeySpec

In [22]:
val SALT: String =
   "jMhKlOuJnM34G6NHkqo9V010GhLAqOpF0BePojHgh1HgNg8^72k"
  
def keyToSpec(key: String): SecretKeySpec = {
  var keyBytes: Array[Byte] = (SALT + key).getBytes("UTF-8")
  val sha: MessageDigest = MessageDigest.getInstance("SHA-1")
  keyBytes = sha.digest(keyBytes)
  keyBytes = util.Arrays.copyOf(keyBytes, 16)
  new SecretKeySpec(keyBytes, "AES")
}

def encrypt(key: String, value: String): String = {
  val cipher: Cipher = Cipher.getInstance("AES/ECB/PKCS5Padding")
  cipher.init(Cipher.ENCRYPT_MODE, keyToSpec(key))
  org.apache.commons.codec.binary.Base64.encodeBase64String(cipher.doFinal(value.getBytes("UTF-8")))
}

def decrypt(key: String, encryptedValue: String): String = {
  val cipher: Cipher = Cipher.getInstance("AES/ECB/PKCS5PADDING")
  cipher.init(Cipher.DECRYPT_MODE, keyToSpec(key))
  new String(cipher.doFinal(org.apache.commons.codec.binary.Base64.decodeBase64(encryptedValue)))
}

SALT: String = jMhKlOuJnM34G6NHkqo9V010GhLAqOpF0BePojHgh1HgNg8^72k
keyToSpec: (key: String)javax.crypto.spec.SecretKeySpec
encrypt: (key: String, value: String)String
decrypt: (key: String, encryptedValue: String)String

In [24]:
val key = "123456789"

key: String = 123456789

In [25]:
import org.apache.spark.sql.functions.lit

import org.apache.spark.sql.functions.lit

In [27]:
val df3 = dfyellow.withColumn("key", lit(key))

df3: org.apache.spark.sql.DataFrame = [eventdatetime: timestamp, customername: string ... 6 more fields]

In [28]:
val encryptUDF = udf(encrypt _)
val decryptUDF = udf(decrypt _)

encryptUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,StringType,Some(List(StringType, StringType)))
decryptUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,StringType,Some(List(StringType, StringType)))

In [29]:
val df4 = df3.withColumn("encryptedcust", encryptUDF(col("key").cast(StringType),col("customername").cast(StringType)))

df4: org.apache.spark.sql.DataFrame = [eventdatetime: timestamp, customername: string ... 7 more fields]

In [30]:
display(df4)

In [31]:
val df5 = df4.withColumn("deencryptedcust", decryptUDF(col("key").cast(StringType),col("encryptedcust").cast(StringType)))

df5: org.apache.spark.sql.DataFrame = [eventdatetime: timestamp, customername: string ... 8 more fields]

In [32]:
display(df5)